In [1]:
#import library
import numpy as np
import pandas as pd

In [2]:
#read data
train_data = pd.read_csv("train.csv")

In [3]:
# 국가를 대륙으로 매핑하는 딕셔너리
country_continent_dict = {
 'US': 'US',
 'Cuba': 'North America',
 'Portugal': 'Europe',
 'Mexico': 'North America',
 'Unknown': 'US',
 'Puerto-Rico': 'North America',
 'Germany': 'Europe',
 'Japan': 'Asia',
 'Poland': 'Europe',
 'Columbia': 'South America',
 'Philippines': 'Asia',
 'Italy': 'Europe',
 'Trinadad&Tobago': 'South America',
 'England': 'Europe',
 'South Korea': 'Asia',
 'Iran': 'Asia',
 'France': 'Europe',
 'India': 'Asia',
 'China': 'Asia',
 'Dominican-Republic': 'North America',
 'Scotland': 'Europe',
 'Ecuador': 'South America',
 'Nicaragua': 'North America',
 'Peru': 'South America',
 'Cambodia': 'Asia',
 'Canada': 'North America',
 'Jamaica': 'North America',
 'Vietnam': 'Asia',
 'Hong Kong': 'Asia',
 'Thailand': 'Asia',
 'Haiti': 'North America',
 'Guatemala': 'North America',
 'Laos': 'Asia',
 'Yugoslavia': 'Europe',
 'Ireland': 'Europe',
 'El-Salvador': 'North America',
 'Panama': 'North America',
 'Honduras': 'North America',
 'Greece': 'Europe',
 'Outlying-U S (Guam USVI etc)': 'US',
 'Hungary': 'Europe',
 'Taiwan': 'Asia',
 'Holand-Netherlands': 'Europe'
}

# 업데이트할 열 목록: 본인 출신국가, 엄마 출신국가, 아빠 출신국가
columns_to_update = ['Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)']

# 각 열에 대해 국가를 대륙으로 매핑
for column in columns_to_update:
    train_data[column] = train_data[column].map(country_continent_dict)

# Remove rows where 'Age' is below 17 or above 75
train_data = train_data[train_data['Age'].between(17, 75)]

# Remove rows with 'Employment Status' as 'Not Working' or 'Seeking Full-Time'
train_data = train_data[~train_data['Employment_Status'].isin(['Not Working', 'Seeking Full-Time'])]

# Drop the 'Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status' columns
train_data.drop(['Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status'], axis=1, inplace=True)

# Map 'Gender' values from 'M' and 'F' to 0 and 1, respectively
train_data['Gender'] = train_data['Gender'].map({'M': 0, 'F': 1})


## 내가 추가 income이상치 제거
train_data = train_data[train_data['Income'] <= 2500]


# Consolidate education levels and rename as specified
education_map = {
    'High graduate': 'High', 'High Senior': 'High', 
    'High Junior': 'High', 'High Sophomore': 'High',
    'Elementary (5-6)': 'Elementary(1-6)', 'Elementary (1-4)': 'Elementary(1-6)',
    'Kindergarten': 'Baby', 'Children': 'Baby'
}
train_data['Education_Status'] = train_data['Education_Status'].replace(education_map)
columns=[
    'Education_Status',
    'Employment_Status',
    'Industry_Status',
    'Occupation_Status',
    'Race',
    'Hispanic_Origin',
    'Martial_Status',
    'Household_Summary',
    'Citizenship',
    'Birth_Country',
    'Birth_Country (Father)',
    'Birth_Country (Mother)',
    'Tax_Status'
]
edit_train_data=pd.get_dummies(train_data,columns=columns,dtype=int)
edit_train_data = edit_train_data.drop(columns=['ID'])
edit_train_data = edit_train_data.drop(columns=['Income'])
edit_train_data.to_csv("editedTrain.csv", index=False)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 데이터 불러오기 (실제 데이터로 대체 필요)
X = edit_train_data
y = train_data['Income']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 생성
lgbm_model = LGBMRegressor()

# 랜덤 서치를 위한 파라미터 분포 설정
param_dist = {
    'learning_rate': np.logspace(-3, 0, 100),  # 로그 스케일로 학습률 분포 설정
    'n_estimators': [100, 200, 300, 400, 500],  # 트리의 수
    'max_depth': [3, 5, 7, 9],  # 트리의 최대 깊이
    'num_leaves': [15, 31, 63, 127],  # 각 트리의 최대 잎의 수
    'min_child_samples': [10, 20, 30, 50],  # 각 리프 노드에 필요한 최소 데이터 수
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  # 샘플링 비율
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]  # 각 트리를 구성하는 특성의 비율
}

# 랜덤 서치 수행
random_search = RandomizedSearchCV(
    estimator=lgbm_model,
    param_distributions=param_dist,
    n_iter=100,  # 시도할 하이퍼파라미터 조합의 수
    cv=5,  # 교차 검증 폴드 수
    scoring='neg_mean_squared_error',  # 평가 지표
    n_jobs=-1,  # 모든 CPU 코어 사용
    verbose=2,  # 실행 과정 표시
    random_state=42  # 랜덤 시드 고정
)

# 랜덤 서치 수행
random_search.fit(X_train, y_train)

# 최적의 모델 및 파라미터 출력
print("Best Parameters:", random_search.best_params_)

# 테스트 데이터에 대한 예측 수행 및 성능 평가
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R²:", r2)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003029 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 335
[LightGBM] [Info] Number of data points in the train set: 12528, number of used features: 113
[LightGBM] [Info] Start training from score 646.940693
Best Parameters: {'subsample': 1.0, 'num_leaves': 31, 'n_estimators': 200, 'min_child_samples': 10, 'max_depth': 9, 'learning_rate': 0.030538555088334154, 'colsample_bytree': 0.7}
[LightGBM

In [5]:
testData = pd.read_csv("EditedTest.csv")

In [6]:

row_to_zero = testData[(testData["Age"] < 17) | 
                      (testData["Age"] > 75) | 
                      (testData["Employment_Status"] == "Not Working") | 
                      (testData["Employment_Status"] == "Seeking Full-Time")].index

# 'df_encoded'는 훈련 데이터에 대해 이미 원핫 인코딩이 수행된 DataFrame입니다.
# 'train_cols'는 원핫 인코딩된 훈련 데이터의 열 순서입니다.
train_cols = edit_train_data.columns

# 'columns' 리스트에 지정된 열에 대해 testData 데이터프레임을 원핫 인코딩합니다.
edited_testData = pd.get_dummies(testData, columns=columns, dtype=int)

# 훈련 데이터에만 있는 열을 찾습니다.
missing_cols = set(train_cols) - set(edited_testData.columns)

# 훈련 데이터에만 있는 열을 테스트 데이터에 추가하고, 해당 열의 값을 0으로 설정합니다.
for col in missing_cols:
    edited_testData[col] = 0

# 테스트 데이터에서 훈련 데이터에 없는 열을 찾습니다.
extra_cols = set(edited_testData.columns) - set(train_cols)

# 테스트 데이터에서 훈련 데이터에 없는 열을 제거합니다.
edited_testData = edited_testData.drop(columns=extra_cols)

# 테스트 데이터의 열 순서를 훈련 데이터의 열 순서와 동일하게 재정렬합니다.
edited_testData = edited_testData[train_cols]

# 이제 'edited_testData'는 원핫 인코딩된 테스트 데이터이며, 열 순서가 'df_encoded'와 일치합니다.


In [7]:
y_pred_lightGBM = best_model.predict(edited_testData) 
for i in range(len(y_pred_lightGBM)):
    if i in row_to_zero:
        y_pred_lightGBM[i]=0
y_pred_lightGBM[y_pred_lightGBM < 0] = 0

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [8]:
# ID 생성
ids = [f'TEST_{i:04d}' for i in range(len(y_pred_lightGBM))]

# 데이터프레임 생성
df = pd.DataFrame({
    'ID': ids,
    'Income': y_pred_lightGBM
})

# CSV 파일로 저장
df.to_csv('submissionFilelightGBM.csv', index=False)